In [2]:
from glob import glob
from astroquery.mast import Mast, Observations
import os
import shutil
import numpy as np
import json
import requests
import asdf
from astropy import log
from astropy.io import ascii, fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.utils.data import download_file
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch, LinearStretch
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.table import Table
import astropy.units as u
import stdatamodels


# do this before importing webb
os.environ["CRDS_PATH"] = "/orange/adamginsburg/jwst/brick/crds/"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"

from jwst.pipeline import calwebb_image3
from jwst.pipeline import Detector1Pipeline, Image2Pipeline

# Individual steps that make up calwebb_image3
from jwst.tweakreg import TweakRegStep
from jwst.tweakreg.utils import adjust_wcs
from jwst.skymatch import SkyMatchStep
from jwst.outlier_detection import OutlierDetectionStep
from jwst.resample import ResampleStep
from jwst.source_catalog import SourceCatalogStep
from jwst import datamodels
from jwst.associations import asn_from_list
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base
from jwst.datamodels import DataModel
from jwst.datamodels import ImageModel

import jwst.datamodels

In [12]:
files = glob("/orange/adamginsburg/jwst/cloudc/F405N/pipeline/*nrc*long_destreak.fits")
files

['/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_08201_00004_nrcblong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_02201_00001_nrcblong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_08201_00001_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_08201_00002_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_08201_00002_nrcblong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_08201_00001_nrcblong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_02201_00003_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002002_02201_00002_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_08201_00004_nrcalong_destreak.fits',
 '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/jw02221002001_08201_00003_nrcalong_destre

In [13]:
print('test')

test


In [14]:
af = stdatamodels.jwst.datamodels.open(files[1])
meta = af['meta']
wcs = meta.wcs
wcs

<WCS(output_frame=world, input_frame=detector, forward_transform=Model: CompoundModel
Inputs: ('x0', 'x1')
Outputs: ('lon', 'lat')
Model set size: 1
Expression: [0] & [1] | [2] & [3] | [4] & [5] | [6] | [7] & [8] | [9] | [10] & [11] | [12] & [13] | [14] & [15] | [16] & [17] | [18] & [19] | [20] | [21] | [22]
Components: 
    [0]: <Shift(offset=-0.132)>

    [1]: <Shift(offset=-0.137)>

    [2]: <Shift(offset=1.)>

    [3]: <Shift(offset=1.)>

    [4]: <Shift(offset=-1024.5)>

    [5]: <Shift(offset=-1024.5)>

    [6]: <Mapping((0, 1, 0, 1))>

    [7]: <Polynomial2D(5, c0_0=0., c1_0=0.06285605, c2_0=-0.00000012, c3_0=0., c4_0=0., c5_0=-0., c0_1=-0., c0_2=0.0000001, c0_3=0., c0_4=0., c0_5=-0., c1_1=-0.00000074, c1_2=0., c1_3=-0., c1_4=0., c2_1=0., c2_2=0., c2_3=-0., c3_1=-0., c3_2=0., c4_1=-0.)>

    [8]: <Polynomial2D(5, c0_0=0., c1_0=0.00008378, c2_0=0.00000031, c3_0=0., c4_0=0., c5_0=-0., c0_1=0.06315358, c0_2=-0.00000044, c0_3=0., c0_4=-0., c0_5=0., c1_1=-0.00000023, c1_2=0., c1_3=-0

In [15]:
wcs.pixel_to_world(50, 50)

<SkyCoord (ICRS): (ra, dec) in deg
    (266.59927065, -28.62643857)>

In [16]:
wcs_new.pixel_to_world(50, 50)

NameError: name 'wcs_new' is not defined

In [ ]:
xshift = 7.9*u.arcsec
yshift = -0.3*u.arcsec

In [17]:
wcs_new = adjust_wcs(wcs, delta_ra = -yshift, delta_dec = -xshift)
wcs_new

<WCS(output_frame=world, input_frame=detector, forward_transform=Model: CompoundModel
Inputs: ('x0', 'x1')
Outputs: ('lon', 'lat')
Model set size: 1
Expression: [0] & [1] | [2] & [3] | [4] & [5] | [6] | [7] & [8] | [9] | [10] & [11] | [12] & [13] | [14] & [15] | [16] & [17] | [18] & [19] | [20] | [21] | [22]
Components: 
    [0]: <Shift(offset=-0.132)>

    [1]: <Shift(offset=-0.137)>

    [2]: <Shift(offset=1.)>

    [3]: <Shift(offset=1.)>

    [4]: <Shift(offset=-1024.5)>

    [5]: <Shift(offset=-1024.5)>

    [6]: <Mapping((0, 1, 0, 1))>

    [7]: <Polynomial2D(5, c0_0=0., c1_0=0.06285605, c2_0=-0.00000012, c3_0=0., c4_0=0., c5_0=-0., c0_1=-0., c0_2=0.0000001, c0_3=0., c0_4=0., c0_5=-0., c1_1=-0.00000074, c1_2=0., c1_3=-0., c1_4=0., c2_1=0., c2_2=0., c2_3=-0., c3_1=-0., c3_2=0., c4_1=-0.)>

    [8]: <Polynomial2D(5, c0_0=0., c1_0=0.00008378, c2_0=0.00000031, c3_0=0., c4_0=0., c5_0=-0., c0_1=0.06315358, c0_2=-0.00000044, c0_3=0., c0_4=-0., c0_5=0., c1_1=-0.00000023, c1_2=0., c1_3=-0

In [18]:
wcs == wcs_new

False

In [19]:
wcs == ww

False

In [7]:
filename = files[0]
align_image = filename.replace("_destreak.fits", "_align.fits")
shutil.copy(filename, align_image)
offsets_tbl = Table.read('/orange/adamginsburg/jwst/cloudc/offsets/Offsets_JWST_Cloud_C.csv')
row = offsets_tbl[filename.split('/')[-1] == offsets_tbl['Filename_1']]
print('Running manual align.')
try:
    xshift = float(row['xshift (arcsec)'])*u.arcsec
    yshift = float(row['yshift (arcsec)'])*u.arcsec
except:
    log.info('Something went wrong with manual align, running default values.')
    visit = filename.split('_')[0][-3:]
    if visit == '001':
        xshift = 8*u.arcsec
        yshift = -0.3*u.arcsec
    elif visit == '002':
        xshift = 3.9*u.arcsec
        yshift = 1.5*u.arcsec
    else:
        xshift = 0*u.arcsec
        yshift = 0*u.arcsec
align_fits = ImageModel(align_image)
ww = adjust_wcs(align_fits.meta.wcs, delta_ra = yshift, delta_dec = xshift)
align_fits.meta.wcs = ww
align_fits.save(align_image)
align_fits = fits.open(align_image)
align_fits[1].header.update(ww.to_fits()[0])
align_fits.writeto(align_image, overwrite=True)
filename = align_image

2023-12-05 12:40:42,841 - stpipe - INFO - Something went wrong with manual align, running default values.


Running manual align.
INFO: Something went wrong with manual align, running default values. [unknown]


In [ ]:
'''
for f in files:
    align_image = member['expname'].replace("_destreak.fits", "_align.fits")
    shutil.copy(member['expname'], align_image)
    offsets_tbl = Table.read('/orange/adamginsburg/jwst/cloudc/offsets/Offsets_JWST_Cloud_C.csv')
    row = offsets_tbl[member['expname'].split('/')[-1] == offsets_tbl['Filename_1']]
    print('Running manual align.')
    try:
        xshift = float(row['xshift (arcsec)'])*u.arcsec
        yshift = float(row['yshift (arcsec)'])*u.arcsec
    except:
        log.info('Something went wrong with manual align, running default values.')
        visit = member['expname'].split('_')[0][-3:]
        if visit == '001':
            xshift = 8*u.arcsec
            yshift = -0.3*u.arcsec
        elif visit == '002':
            xshift = 3.9*u.arcsec
            yshift = 1*u.arcsec
        else:
            xshift = 0*u.arcsec
            yshift = 0*u.arcsec
    align_fits = ImageModel(align_image)
    ww = adjust_wcs(align_fits.meta.wcs, delta_ra = yshift, delta_dec = xshift)
    align_fits.meta.wcs = ww
    align_fits.save(align_image)
    align_fits = fits.open(align_image)
    align_fits[1].header.update(ww.to_fits()[0])
    align_fits.writeto(align_image, overwrite=True)
'''

In [22]:
for f in files:
    filename = f
    align_image = filename.replace("_destreak.fits", "_align.fits")
    shutil.copy(filename, align_image)
    offsets_tbl = Table.read('/orange/adamginsburg/jwst/cloudc/offsets/Offsets_JWST_Cloud_C.csv')
    row = offsets_tbl[filename.split('/')[-1] == offsets_tbl['Filename_1']]
    print('Running manual align.')
    try:
        xshift = float(row['xshift (arcsec)'])*u.arcsec
        yshift = float(row['yshift (arcsec)'])*u.arcsec
    except:
        log.info('Something went wrong with manual align, running default values.')
        visit = filename.split('_')[0][-3:]
        if visit == '001':
            xshift = 8*u.arcsec
            yshift = -0.3*u.arcsec
        elif visit == '002':
            xshift = 3.9*u.arcsec
            yshift = 1*u.arcsec
        else:
            xshift = 0*u.arcsec
            yshift = 0*u.arcsec
    align_fits = ImageModel(align_image)
    ww = adjust_wcs(align_fits.meta.wcs, delta_ra = yshift+0.6*u.arcsec, delta_dec = xshift)
    align_fits.meta.wcs = ww
    align_fits.save(align_image)
    align_fits = fits.open(align_image)
    align_fits[1].header.update(ww.to_fits()[0])
    align_fits.writeto(align_image, overwrite=True)
    #filename = align_image

Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.
Running manual align.


In [9]:
print('done')

done


In [43]:
models = jwst.datamodels.open(glob("/orange/adamginsburg/jwst/cloudc/F405N/pipeline/*nrc*long_align.fits"))

In [44]:
abs_refcat = '/orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-F405N-merged_vvvcat.ecsv'

In [45]:
tweakreg = TweakRegStep()
tweakreg.fitgeometry = 'general'
tweakreg.brightest = 500
tweakreg.snr_threshold = 30
tweakreg.abs_refcat = abs_refcat
tweakreg.save_catalogs = True
tweakreg.catalog_format = 'ecsv'
#tweakreg.kernel_fwhm = fwhm_pix
tweakreg.nclip = 5
tweakreg.sharplo = 0.3
tweakreg.sharphi = 0.9
tweakreg.roundlo = -0.25
tweakreg.roundhi = 0.25
tweakreg.separation = 0.5
tweakreg.output_dir = "/orange/adamginsburg/jwst/cloudc/F405N/pipeline/"#output_dir
tweakreg.save_results = True
tweakreg.skip = False
tweakreg.searchrad = 5

tweak = tweakreg.run(models)


2023-11-17 21:53:13,053 - stpipe.TweakRegStep - INFO - TweakRegStep instance created.
2023-11-17 21:53:13,722 - stpipe.TweakRegStep - INFO - Step TweakRegStep running with args (<ModelContainer>,).
2023-11-17 21:53:13,727 - stpipe.TweakRegStep - INFO - Step TweakRegStep parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': '/orange/adamginsburg/jwst/cloudc/F405N/pipeline/', 'output_ext': '.fits', 'output_use_model': True, 'output_use_index': True, 'save_results': True, 'skip': False, 'suffix': None, 'search_output_file': True, 'input_dir': '', 'save_catalogs': True, 'use_custom_catalogs': False, 'catalog_format': 'ecsv', 'catfile': '', 'kernel_fwhm': 2.5, 'snr_threshold': 30, 'sharplo': 0.3, 'sharphi': 0.9, 'roundlo': -0.25, 'roundhi': 0.25, 'brightest': 500, 'peakmax': None, 'bkg_boxsize': 400, 'enforce_user_order': False, 'expand_refcat': False, 'minobj': 15, 'searchrad': 5, 'use2dhist': True, 'separation': 0.5, 'tolerance': 0.7, 'xoffset': 0.0, 'yoff

In [46]:
print('done')

done
